## `Formula and structural predictions of the preprocessed data`

Import libraries:

In [ ]:
import os
import glob
from pyopenms import *
import shutil

#### `1) SIRIUS Adapter`
The SIRIUS function is optional and includes the SIRIUS Adapter Algorithm from the Boecher lab. 


The algorithm generates formula predictions from scores calculated from 1) MS2 fragmentation scores (ppm error + intensity) and 2) MS1 isotopic pattern scores.


It can only compute feautures that are singly charged. There is also a compound timeout so that it doesn"t compute for longer than 100 seconds per feature, which normally happens with larger molecules.
        
>-sirius:compound_timeout <number>                    

>Maximal computation time in seconds for a single compound. 0 for an infinite amount of time. (default: "100" min: "0")

#### `2) CSI:FingerID`

The CSI_fingerID function is another algorithm from the Boecher lab, just like SIRIUS adapter and is using the formula predictions from SIRIUS, to search in structural libraries and predict the structure of each formula

If you replace: 
    
>out_csifingerid = os.path.join("results", "", "interim", "", "CSI", "", "structures_" + os.path.basename(filename) +".mzTab")

with an empty string:

>out_csifingerid = ""
        
CSI:FingerID will be ignored and will not be computed.

###### Documentation: https:""boecker-lab.github.io"docs.sirius.github.io"

###### Citation: Kai Dührkop, Huibin Shen, Marvin Meusel, Juho Rousu, and Sebastian Böcker, Searching molecular structure databases with tandem mass spectra using CSI:FingerID, PNAS October 13, 2015 112 (41) 12580-12585, https:""doi.org"10.1073"pnas.1509788112

In [ ]:
path = os.path.join("results", "interim", "SiriusCSI")
if not os.path.exists(path): # if it doesn't exist
    os.mkdir(path) # create a path directory

feature_files = sorted(glob.glob(os.path.join("results", "interim", "Preprocessing", "MFD_*.featureXML")))
input_mzml_files = sorted(glob.glob(os.path.join("results", "interim", "Preprocessing", "MapAligned_*.mzML")))

feature_maps = []
for file in feature_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(file, fmap)
    feature_maps.append(fmap)

for filename in input_mzml_files:
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    print(exp.getNrSpectra())
    
    for fmap in feature_maps:
        if os.path.basename(fmap.getMetaValue("spectra_data")[0].decode())[7:] == os.path.basename(filename)[11:]:
            fm_no_sub = FeatureMap(fmap)
            fm_no_sub.clear(False)
            for f in fmap:
                f.setConvexHulls([])
                f.setSubordinates([])
                fm_no_sub.push_back(f)
            
            featureinfo = os.path.join(path, 'MFD_ncv_' + os.path.basename(filename)[11:-5] + ".featureXML")
            FeatureXMLFile().store(featureinfo, fm_no_sub)
        
            sirius_algo = SiriusAdapterAlgorithm()
            sirius_algo_par = sirius_algo.getDefaults()
            sirius_algo_par.setValue("preprocessing:filter_by_num_masstraces", 2) #Number of mass traces each feature has to have to be included
            sirius_algo_par.setValue("preprocessing:feature_only", "true") #Uses the feature information from in_fm_no_sub to reduce the search space to MS2
            sirius_algo_par.setValue("sirius:profile", "orbitrap")
            sirius_algo_par.setValue("sirius:db", "none")
            sirius_algo_par.setValue("sirius:ions_considered", "[M+H]+, [M-H2O+H]+, [M+Na]+, [M+NH4]+")
            sirius_algo_par.setValue("sirius:elements_enforced", "CHN[15]OS[4]Cl[2]P[2]") 
            sirius_algo_par.setValue("project:processors", 2)
            sirius_algo_par.setValue("fingerid:db", "BIO")
            sirius_algo.setParameters(sirius_algo_par)
                
            fm_info = FeatureMapping_FeatureMappingInfo()
            feature_mapping = FeatureMapping_FeatureToMs2Indices() 
            sirius_algo.preprocessingSirius(featureinfo,
                                            exp,
                                            fm_info,
                                            feature_mapping)
            sirius_algo.logFeatureSpectraNumber(featureinfo, 
                                                feature_mapping,
                                                exp)
            msfile = SiriusMSFile()
            debug_level = 3
            sirius_tmp = SiriusTemporaryFileSystemObjects(debug_level)
            siriusstring= String(sirius_tmp.getTmpMsFile())
            feature_only = sirius_algo.isFeatureOnly()
            isotope_pattern_iterations = sirius_algo.getIsotopePatternIterations()
            no_mt_info = sirius_algo.isNoMasstraceInfoIsotopePattern()
            compound_info = []
            msfile.store(exp,
                        String(sirius_tmp.getTmpMsFile()),
                        feature_mapping, 
                        feature_only,
                        isotope_pattern_iterations, 
                        no_mt_info, 
                        compound_info)

            out_csifingerid = os.path.join(path, "structures_" + os.path.basename(filename)[11:-5] +".mzTab")
            executable = shutil.which("sirius")
            subdirs = sirius_algo.callSiriusQProcess(String(sirius_tmp.getTmpMsFile()),
                                                    String(sirius_tmp.getTmpOutDir()),
                                                    String(executable),
                                                    String(out_csifingerid),
                                                    False)
            candidates = sirius_algo.getNumberOfSiriusCandidates()
            sirius_result = MzTab()
            siriusfile = MzTabFile()
            SiriusMzTabWriter.read(subdirs,
                                filename,
                                candidates,
                                sirius_result)

            sirius_file= os.path.join(path,"formulas_" + os.path.basename(filename)[11:-5] +".mzTab")
            siriusfile.store(sirius_file, sirius_result)

##### Convert mzTab files to dataframes for visualisation

In [ ]:
import pyteomics
from pyteomics import mztab

##### Explanation of columns for SIRIUS
###### - mz= mass-to-charge ratio (m"z)
###### - RT= retention time (sec)
###### - intensity = intensity of the feature (AU-arbitrary units)
###### - FWHM= Full Width of the peak at Half its Maximum height
###### - num_of_masstraces	= number of mass traces detected (single mass traces are excluded). This is relevant to the isotopic pattern
###### - isotope_distances = distance in mz between the isotopes (jumps of app. 1 is important to confirm that this is a real feature) 

In [ ]:
path = os.path.join("results", "Sirius")
if not os.path.exists(path): # if it doesn't exist
    os.mkdir(path) # create a path directory

input_SIRIUS_files = glob.glob(os.path.join("results", "interim", "SiriusCSI", "formulas_*.mzTab"))
for filename in input_SIRIUS_files:
    sirius=  pyteomics.mztab.MzTab(filename, encoding="UTF8", table_format="df")
    sirius.metadata
    df= sirius.small_molecule_table
    SIRIUS_DF= df.drop(columns= ["identifier", "smiles", "inchi_key", "description", "calc_mass_to_charge", "charge", "taxid", "species","database", "database_version", "spectra_ref", "search_engine", "modifications"])
    SIRIUS_DF=SIRIUS_DF[SIRIUS_DF["opt_global_explainedIntensity"] >= 0.4] #opt_global_explainedIntensity should be higher than 0.8 or 0.9 even for reliable results
    SIRIUS_DF= SIRIUS_DF.sort_values(by= "exp_mass_to_charge")
    SIRIUS_DF= SIRIUS_DF.rename(columns= {"best_search_engine_score[1]":	"SiriusScore"})
    SIRIUS_DF= SIRIUS_DF.rename(columns= {"best_search_engine_score[2]":	"TreeScore"})
    SIRIUS_DF= SIRIUS_DF.rename(columns= {"best_search_engine_score[3]":	"IsotopeScore"})
    SIRIUS_DF=SIRIUS_DF[SIRIUS_DF["IsotopeScore"] > 0.0] 
    sirius_DF_file= os.path.join(path, os.path.basename(filename)[:-6] + ".csv")
    SIRIUS_DF.to_csv(sirius_DF_file)
display(filename, SIRIUS_DF) #example dataframe

In [ ]:
input_CSI_files = glob.glob(os.path.join("results", "interim", "SiriusCSI", "structures_*.mzTab"))
for filename in input_CSI_files:
    CSI=  pyteomics.mztab.MzTab(filename, encoding="UTF8", table_format="df")
    CSI.metadata
    DF= CSI.small_molecule_table
    DF= DF.drop(columns= ["calc_mass_to_charge", "charge", "taxid", "species","database", "database_version", "spectra_ref", "search_engine", "modifications"])
    csifingerID_file= os.path.join(path + os.path.basename(filename)[:-6]+ ".csv")
    DF.to_csv(csifingerID_file)
    display(DF)